In [3]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')


In [4]:
df = pd.read_csv('data/Stud.csv')

In [5]:
x=df.drop(columns=['math score'],axis=1)

In [6]:
y=df['math score']

In [7]:
num_feat=x.select_dtypes(exclude="object").columns
cat_feat=x.select_dtypes(include="object").columns
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()
preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_feat),
        ("StandardScaler",numeric_transformer,num_feat)
    ]
)

In [9]:
X=preprocessor.fit_transform(x)
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape
X_test.shape

(200, 19)

In [17]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    r2_square=r2_score(true,predicted)
    return mae,mse,r2_square

In [18]:
models={
    "Linear Regression":LinearRegression(),
    "Ridge":Ridge(),
    "Lasso":Lasso(),
    "KNN Regressor":KNeighborsRegressor(),
    "Decision Tree":DecisionTreeRegressor(),
    "Random Forest":RandomForestRegressor(),
    "XGB Regressor":XGBRegressor(),
    "CatBoost Regressor":CatBoostRegressor(verbose=False),
    "Ada Boost Regressor":AdaBoostClassifier()
}
model_list=[]
r2_list=[]

In [21]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    mae,mse,r2_square=evaluate_model(y_test,y_test_pred)
    train_mae,train_mse,train_r2_square=evaluate_model(y_train,y_train_pred)
    r2_list.append(r2_square)
    model_list.append(list(models.keys())[i])


In [22]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model','R2_Score']).sort_values(by='R2_Score',ascending=False)

,Model,R2_Score
1,Ridge,0.880593
0,Linear Regression,0.879689
7,CatBoost Regressor,0.851632
5,Random Forest,0.851578
6,XGB Regressor,0.827797
2,Lasso,0.825320
3,KNN Regressor,0.783898
4,Decision Tree,0.755793
8,Ada Boost Regressor,0.213749
